In [1]:
from ccnet_spark.pipe_line import Pipeline, Config,PipelineStep
import time
from pyspark.sql import SparkSession
import sys

pipeline=[
        PipelineStep.REAL_LEN,
        PipelineStep.HASH,
        PipelineStep.DEDUP_KEEP,
        PipelineStep.LID,
        PipelineStep.SP,
        PipelineStep.LM,
        PipelineStep.PP_BUCKET,
        PipelineStep.DROP,
],
spark = (
    SparkSession.builder.appName("CCNETSpark_ONLY")
    # .master("local[*]")
    .config("spark.executor.memory", "100g")
    .config("spark.driver.memory", "100g")
    .config("spark.driver.maxResultSize", "100g")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    # .config("spark.executor.extraJavaOptions", "-XX:+PrintGCDetails -XX:+PrintGCDateStamps -XX:+PrintGCTimeStamps")
    # .config("spark.driver.extraJavaOptions", "-XX:+PrintGCDetails -XX:+PrintGCDateStamps -XX:+PrintGCTimeStamps")
    .getOrCreate()
)
config = Config(
        isSample=True,
        n_segments=1,
        sampleRate=0.01,
        cache_dir="../../cached_data/",
        output_dir="../../cached_data/",
        fasttext_model_path="../../cc_net/bin/lid.bin",
        lm_dir="../../cc_net/data/lm_sp",
        cutoff_csv_path="../../cc_net/cc_net/data/cutoff.csv",
        dump="2019-18",
        pipeline=pipeline,
    )
pipeline = Pipeline(config,spark)
df = pipeline.load_data()

24/04/12 13:47:48 WARN DataSource: All paths were ignored:
  file:/root/wxl_folder/cached_data/sdf_parquet/2019-18/_sampleRate_1_segment_0_min_len_300.parquet
2024-04-12 13:47 INFO 2070853:root - load segment 0, with sampleRate:1.0%,min_len:300,with date:2019-18


In [2]:
print(df.columns)
df.select("url","length","nlines","raw_content","cc_segment").show(5)

['url', 'date_download', 'digest', 'length', 'nlines', 'source_domain', 'title', 'raw_content', 'cc_segment']
+--------------------+------+------+--------------------------------+--------------------+
|                 url|length|nlines|                     raw_content|          cc_segment|
+--------------------+------+------+--------------------------------+--------------------+
|http://rgdv.com.c...|  2286|    63|  更多\n更多\n全站搜索\n更多\...|CC-MAIN-201904181...|
|https://www.place...|  4964|   141|            Explore\nLife\nFo...|CC-MAIN-201904181...|
|http://voidslave....|369505|  7353|            sample lunch and ...|CC-MAIN-201904181...|
|https://inaba-afi...| 14478|   260|年中行事\nこどもの日\n母の日\...|CC-MAIN-201904181...|
|https://www.wamp....|  1694|    41|            Skip to main cont...|CC-MAIN-201904181...|
+--------------------+------+------+--------------------------------+--------------------+
only showing top 5 rows



In [3]:
pipeline.run_step(PipelineStep.REAL_LEN)
print(pipeline.df.columns)
pipeline.df.select("url","length","nlines","raw_content","cc_segment").show(5)

['url', 'date_download', 'digest', 'length', 'nlines', 'source_domain', 'title', 'raw_content', 'cc_segment']
+--------------------+------+------+--------------------------------+--------------------+
|                 url|length|nlines|                     raw_content|          cc_segment|
+--------------------+------+------+--------------------------------+--------------------+
|http://rgdv.com.c...|  1054|    63|  更多\n更多\n全站搜索\n更多\...|CC-MAIN-201904181...|
|https://www.place...|  4898|   141|            Explore\nLife\nFo...|CC-MAIN-201904181...|
|http://voidslave....|369489|  7353|            sample lunch and ...|CC-MAIN-201904181...|
|https://inaba-afi...|  5299|   260|年中行事\nこどもの日\n母の日\...|CC-MAIN-201904181...|
|https://www.wamp....|  1672|    41|            Skip to main cont...|CC-MAIN-201904181...|
+--------------------+------+------+--------------------------------+--------------------+
only showing top 5 rows



In [4]:
pipeline.run_step(PipelineStep.HASH)
print(pipeline.df.columns)
pipeline.df.select("url","length","nlines","raw_content","cc_segment",'exploded_content', 'hash_value').show(5)

['url', 'date_download', 'digest', 'length', 'nlines', 'source_domain', 'title', 'raw_content', 'cc_segment', 'exploded_content', 'hash_value']
+--------------------+------+------+------------------------------+--------------------+----------------+--------------------+
|                 url|length|nlines|                   raw_content|          cc_segment|exploded_content|          hash_value|
+--------------------+------+------+------------------------------+--------------------+----------------+--------------------+
|http://rgdv.com.c...|  1054|    63|更多\n更多\n全站搜索\n更多\...|CC-MAIN-201904181...|       {0, 更多}|[9B 0C 6C 78 58 B...|
|http://rgdv.com.c...|  1054|    63|更多\n更多\n全站搜索\n更多\...|CC-MAIN-201904181...|       {1, 更多}|[9B 0C 6C 78 58 B...|
|http://rgdv.com.c...|  1054|    63|更多\n更多\n全站搜索\n更多\...|CC-MAIN-201904181...|   {2, 全站搜索}|[97 B8 03 99 98 C...|
|http://rgdv.com.c...|  1054|    63|更多\n更多\n全站搜索\n更多\...|CC-MAIN-201904181...|       {3, 更多}|[9B 0C 6C 78 58 B...|
|http://rgdv.com.

In [5]:
pipeline.run_step(PipelineStep.DEDUP_KEEP)
print(pipeline.df.columns)
pipeline.df.select("url","length","nlines","raw_content","cc_segment",'raw_line_id','original_length','original_nlines').show(5)

['digest', 'url', 'date_download', 'source_domain', 'cc_segment', 'original_length', 'original_nlines', 'title', 'nlines', 'raw_content', 'raw_line_id', 'length']


+--------------------+------+------+--------------------+--------------------+--------------------+---------------+---------------+
|                 url|length|nlines|         raw_content|          cc_segment|         raw_line_id|original_length|original_nlines|
+--------------------+------+------+--------------------+--------------------+--------------------+---------------+---------------+
|http://ecrack1.ui...| 42384|    64|Ring ring\n1234 6...|CC-MAIN-201904181...|[0, 1, 2, 3, 4, 5...|          44461|            108|
|https://romankhon...|  2943|   124|رفتن به مطلب\nفرو...|CC-MAIN-201904181...|[0, 1, 2, 3, 4, 5...|           3655|            189|
|https://www.obesi...|  1597|    71|Navigation\nBaria...|CC-MAIN-201904181...|[0, 1, 2, 3, 4, 5...|           3519|            185|
|http://publicrepo...|  7398|    72|Reports on Long-T...|CC-MAIN-201904181...|[0, 4, 6, 7, 8, 9...|           7742|             96|
|http://www.e-isla...|  4531|    38|e-Ислам\nНовостно...|CC-MAIN-201904181..

In [16]:
from pyspark.sql import functions as F
pipeline.df.withColumn("raw_content_len", F.length(pipeline.df["raw_content"])).withColumn("raw_line_id_len", F.size(pipeline.df["raw_line_id"]))

DataFrame[digest: string, url: string, date_download: string, source_domain: string, cc_segment: string, original_length: int, original_nlines: bigint, title: string, nlines: bigint, raw_content: string, raw_line_id: array<int>, length: int, raw_content_len: int, raw_line_id_len: int]

In [17]:
pipeline.df.select("raw_line_id").show()

+--------------------+
|         raw_line_id|
+--------------------+
|[0, 1, 2, 3, 4, 5...|
|[0, 1, 2, 3, 4, 5...|
|[0, 1, 2, 3, 4, 5...|
|[0, 4, 6, 7, 8, 9...|
|[0, 1, 2, 3, 5, 6...|
|[0, 1, 2, 3, 4, 5...|
|[0, 3, 4, 5, 6, 9...|
|[0, 1, 4, 7, 10, ...|
|[2, 3, 4, 6, 7, 8...|
|[2, 3, 4, 5, 6, 7...|
|[0, 1, 3, 4, 5, 7...|
|[0, 1, 3, 4, 6, 7...|
|[0, 2, 3, 4, 5, 6...|
|[0, 1, 2, 3, 4, 6...|
|[0, 1, 3, 4, 5, 6...|
|[1, 2, 4, 5, 6, 7...|
|[0, 1, 2, 3, 4, 8...|
|[0, 1, 2, 4, 5, 6...|
|[0, 1, 2, 3, 4, 5...|
|[0, 1, 2, 3, 4, 5...|
+--------------------+
only showing top 20 rows



In [15]:
pipeline.df.withColumn("raw_line_id_len", F.size(pipeline.df["raw_line_id"]))


DataFrame[digest: string, url: string, date_download: string, source_domain: string, cc_segment: string, original_length: int, original_nlines: bigint, title: string, nlines: bigint, raw_content: string, raw_line_id: array<int>, length: int, raw_line_id_len: int]